<a href="https://colab.research.google.com/github/nayalafifi/data-feature/blob/main/Project1_DataFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import requests

NEWS_API = userdata.get('NEWS_API')
FORMULA1_API = userdata.get("FORMULA1_API")

if NEWS_API:#checking if the keys were retrived for both APIS
  print("NEWS_API has been received")
else:
  print("NEWS_API has not been received")

if FORMULA1_API:
  print("FORMULA1_API has been received")
else:
  print("FORMULA1_API has not been received")


NEWS_API has been received
FORMULA1_API has been received


In [ ]:
#function to find the driver id based on the driver's name input by the user
def find_driver_id(driver_name):
    driver_name_lower = driver_name.lower().strip()#normalize the driver name
    current_season = 2024#adjust to the current season
    url = f"https://ergast.com/api/f1/{current_season}/drivers.json"#url to fetch drivers from current season
    response = requests.get(url)#send request to ergast api

    if response.status_code != 200:#check if the request was successful
        print(f"error: received status code {response.status_code}")
        return None

    driver_data = response.json()
    drivers = driver_data['MRData']['DriverTable']['Drivers']

    for driver in drivers:#find the driver by name
        full_name = f"{driver['givenName']} {driver['familyName']}".lower().strip()#normalize the full name
        if driver_name_lower == full_name:
            return driver['driverId']

    return None

def get_gp_won(driver_id, season):#function to get the last grand prix won by a driver in the current season

    url = f"https://ergast.com/api/f1/{season}/drivers/{driver_id}/results.json?limit=100"
    response = requests.get(url)#send request to get the race results

    if response.status_code != 200:#check if the request was successful
        print(f"error: received status code {response.status_code}")
        return None

    race_data = response.json()#parse the data
    races = race_data['MRData']['RaceTable']['Races']

    for race in reversed(races):#go through races in reverse order, checking for the most recent
        for result in race['Results']:#loop over the results
            if result['position'] == '1':#check if the driver finished 1st
                return {
                    'race_name': race['raceName'],#get the race name (the grand prix)
                    'location': race['Circuit']['Location']['locality'],#get the location of the grand prix
                    'country': race['Circuit']['Location']['country'],#return the country
                    'date': race['date']#return date of the race
                }

    return None

def get_news_articles(location):#our function to get the news article based on the location
    url = f"https://api.thenewsapi.com/v1/news/all?language=en&search={location}&api_token={NEWS_API}"

    response = requests.get(url)#send request to thenewsapi

    if response.status_code != 200:#check if the request was successful
        print(f"error: received status code {response.status_code}")
        return []

    news_data = response.json()
    articles = news_data['data'][:3]#get the first three articles

    news_list = []#store the articles and headlines in an arrary
    for article in articles:
        news_list.append({
            'headline': article['title'],#get the headline from article
            'url': article['url']#get the url from article
        })
    return news_list

def last_gp_won_news_article():#our main function
    driver_name = input("please enter the name of the driver: ")
    driver_id = find_driver_id(driver_name)#get the driver id based on the given name

    if driver_id:
        current_season = 2024
        last_race = get_gp_won(driver_id, current_season)#get the last gp won

        if last_race:
            print(f"{driver_name} last won the {last_race['race_name']} on {last_race['date']} in {last_race['location']}, {last_race['country']}.")

            news_articles = get_news_articles(last_race['location'])#get the news article based on the location

            if news_articles:
                print(f"Below are 3 news articles about {last_race['location']}:")#print of the articles and the headers
                for i, article in enumerate(news_articles, 1):
                    print(f"{i}. {article['headline']} - {article['url']}")
            else:
                print(f"no news articles found for {last_race['location']}.")#else statements to check for error/missing drivers
        else:
            print(f"{driver_name} has not won any grand prix in {current_season}.")
    else:
        print(f"driver '{driver_name}' not found.")


last_gp_won_news_article()#run the main function




please enter the name of the driver: charles leclerc
charles leclerc last won the Italian Grand Prix on 2024-09-01 in Monza, Italy.
Below are 3 news articles about Monza:
1. Silvio Berlusconi’s Monza fires Stroppa as coach - https://www.winnipegfreepress.com/sports/soccer/2022/09/13/silvio-berlusconis-monza-fires-stroppa-as-coach
2. Juventus sign goalkeeper Di Gregorio from Monza - https://www.straitstimes.com/sport/football/juventus-sign-goalkeeper-di-gregorio-from-monza
3. Jehan Daruvala finishes third in F2 sprint race in Italy’s Monza Monza - https://indianexpress.com/article/sports/motor-sport/jehan-daruvala-finishes-third-in-f2-sprint-race-in-italys-monza-monza-8144220/
